In [ ]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fc
import httpx

We are going to build an app that lets people find plumbers, electricians and other specific people to help them with house issues. For this we will use daisy UI components along with a python FastHTML backend.

In [ ]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css')
)

By passing these in as headers (`hdrs`) when creating the app, we ensure this CSS/JS will be available on all pages:

In [ ]:
app = FastHTML(hdrs=daisy_hdrs)
rt = app.route

To see things with these styles applied, we'll make a little conveneince function `preview` that shows them with the app's headers, and later lets us preview routes the same way. In the future, `get_preview` will be available in dialoghelper.

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

Here's how we might make a simple component:

In [ ]:
c = Div(P('hi'), style='color:red')
print(c)

<div style="color:red"><p>hi</p></div>


And to see it rendered:

In [ ]:
preview(c)

Now let's come back to the initial idea of web app we want to implement. The app should let people find plumbers, electricians and other task workers to help them with house issues, similar to uber, airbnb and these kinds of apps. First let's build a nice UI(using daisy components like above), then we visualize it here in the notebook in order to maybe improve it step by step. This basic UI for now will be only for the user that searches task workers in the app, the main view of the app.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great idea! Let's build this step by step.

For the main user view, we'll probably want:
1. A **navbar** with the app name/logo
2. A **search bar** to find services
3. **Category cards** for quick access (Plumber, Electrician, etc.)

Which part would you like to start with? Or would you prefer I sketch out a simple version of all three so you can see the overall layout first?

Yes, come up with the code for all 3 of them.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a basic UI combining all three elements:

```python
def home_page():
    navbar = Div(
        Div(
            A("TaskFinder", href="/", cls="btn btn-ghost text-xl"),
            cls="navbar bg-base-100"
        )
    )
    
    search = Div(
        Input(
            type="text",
            placeholder="Search for services...",
            cls="input input-bordered w-full max-w-xl"
        ),
        cls="flex justify-center my-8"
    )
    
    categories = Div(
        Div(
            Div("🔧", cls="text-4xl mb-2"),
            H3("Plumber", cls="card-title"),
            P("Fix leaks, install pipes"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300"
        ),
        Div(
            Div("⚡", cls="text-4xl mb-2"),
            H3("Electrician", cls="card-title"),
            P("Wiring, repairs, installations"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300"
        ),
        Div(
            Div("🎨", cls="text-4xl mb-2"),
            H3("Painter", cls="card-title"),
            P("Interior and exterior painting"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300"
        ),
        cls="grid grid-cols-1 md:grid-cols-3 gap-6 max-w-5xl mx-auto px-4"
    )
    
    return Div(navbar, search, categories)

preview(home_page())
```

Try running this to see the layout! What do you think of the design?

In [ ]:
@rt
def home_page():
    navbar = Div(
        Div(
            A("TaskFinder", href="/", cls="btn btn-ghost text-xl"),
            cls="navbar bg-base-100"
        )
    )
    
    search = Div(
        Input(
            type="text",
            placeholder="Search for services...",
            cls="input input-bordered w-full max-w-xl"
        ),
        cls="flex justify-center my-8"
    )
    
    results_div = Div(id="results", cls="mt-8")

    categories = Div(
        Div(
            Div("🔧", cls="text-4xl mb-2"),
            H3("Plumber", cls="card-title"),
            P("Fix leaks, install pipes"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300",
            hx_get='/get_workers?category=plumber',
            hx_target="#results"
        ),
        Div(
            Div("⚡", cls="text-4xl mb-2"),
            H3("Electrician", cls="card-title"),
            P("Wiring, repairs, installations"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300",
            hx_get='/get_workers?category=electrician',
            hx_target="#results"
        ),
        Div(
            Div("🎨", cls="text-4xl mb-2"),
            H3("Painter", cls="card-title"),
            P("Interior and exterior painting"),
            cls="card bg-base-200 shadow-xl p-6 cursor-pointer hover:bg-base-300",
            hx_get='/get_workers?category=painter',
            hx_target="#results"
        ),
        
        cls="grid grid-cols-1 md:grid-cols-3 gap-6 max-w-5xl mx-auto px-4"
    )
    
    return Div(navbar, search, categories, results_div)

preview(home_page())

It looks great. Now I'd like to configure the 3 divs such that when user clicks, we get from the backend(we will configure a new @rt method to handle this requests) the task workers for that category being clicked. On the backend we will store the workers' details inside a db(sqlite) and, for the start, we will add some mock values in the DB in ordewr to have something rendered in the UI once the user clicks and request is being made.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good plan! Let's break this into steps:

1. **Create the database** with a workers table
2. **Add some mock data** for testing
3. **Add click handlers** to the category cards (using HTMX)
4. **Create a route** that fetches and returns workers for a category

Which step would you like to tackle first? Or would you like me to show you how to set up the database structure?

Yeah, lets detal with db creation and mock population first.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Let's start with the database structure.

For a workers table, we'll need fields like:
- `id` (primary key)
- `name` 
- `category` (plumber, electrician, painter)
- `rating` (maybe 1-5 stars)
- `hourly_rate`
- `description`

Does this structure look good to you, or would you like to add/change any fields?

Looks good.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Here's the code to create the database and table:

```python
db = database('workers.db')
workers = db.t.workers
if workers not in db.t:
    workers.create(id=int, name=str, category=str, rating=float, hourly_rate=int, description=str, pk='id')
```

Try running this. Once that's done, we can add the mock data. Let me know when you're ready for the next step!

In [ ]:
db = database('workers.db')
workers = db.t.workers
if workers not in db.t:
    workers.create(id=int, name=str, category=str, rating=float, location=str, hourly_rate=int, description=str, pk='id')

In [ ]:
mock_workers = [
    dict(name="Ion Popescu", category="plumber", rating=4.8, location="București, Sector 1", hourly_rate=80, description="Specialist în instalații sanitare, 10 ani experiență"),
    dict(name="Maria Ionescu", category="plumber", rating=4.5, location="București, Sector 3", hourly_rate=75, description="Reparații și montaj instalații"),
    dict(name="Andrei Dumitrescu", category="electrician", rating=4.9, location="Craiova", hourly_rate=90, description="Electrician autorizat ANRE, lucrări rezidențiale"),
    dict(name="Elena Constantinescu", category="electrician", rating=4.7, location="București, Sector 2", hourly_rate=85, description="Instalații electrice și tablouri"),
    dict(name="Vasile Georgescu", category="painter", rating=4.6, location="Cluj-Napoca", hourly_rate=70, description="Zugrav profesionist, interior și exterior"),
    dict(name="Ana Popa", category="painter", rating=4.8, location="București, Sector 4", hourly_rate=65, description="Vopsitorie și finisaje decorative"),
]

workers.insert_all(mock_workers)

<Table workers (id, name, category, rating, location, hourly_rate, description)>

In [ ]:
workers(where="category='plumber'")

[{'id': 1,
  'name': 'Ion Popescu',
  'category': 'plumber',
  'rating': 4.8,
  'location': 'București, Sector 1',
  'hourly_rate': 80,
  'description': 'Specialist în instalații sanitare, 10 ani experiență'},
 {'id': 2,
  'name': 'Maria Ionescu',
  'category': 'plumber',
  'rating': 4.5,
  'location': 'București, Sector 3',
  'hourly_rate': 75,
  'description': 'Reparații și montaj instalații'}]

In [ ]:
@rt
def get_workers(category:str):
    '''
       returns a list of workers matching this category and wrapping each worker's details in a stylish Div such that they are being rendered nicely on the UI once the user clicks a specific category
    '''
    worker_list = workers(where=f"category='{category}'")
    
    cards = [
        Div(
            H3(w['name'], cls="text-xl font-bold"),
            H4(w['location'], cls="text-xl"),
            Div(f"⭐ {w['rating']}", cls="text-yellow-500"),
            P(f"${w['hourly_rate']}/hour", cls="text-lg font-semibold text-primary"),
            P(w['description'], cls="text-sm text-gray-600 mt-2"),
            cls="card bg-white shadow-lg p-4 mb-4"
        )
        for w in worker_list
    ]
    
    return Div(*cards, cls="max-w-3xl mx-auto p-4")

Nope, still nothing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's check a few things:

1. When you click a card, do you see any errors in the browser console? (Right-click → Inspect → Console tab)
2. Is the preview iframe or are you viewing it in a separate browser tab at the server URL?

The issue might be that HTMX doesn't work well inside the preview iframe. Try opening the app in a new browser tab using the URL that `serve()` printed out (probably something like `http://localhost:5001`). Does clicking work there?

Nope, still nothing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let me check if there's a route issue. Can you try visiting this URL directly in your browser:

```
http://localhost:5001/get_workers?category=plumber
```

Does that show you the worker cards? If it does, then the route works but HTMX isn't triggering. If it doesn't, we need to fix the route setup.

Nope, still nothing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's strange - if the route itself isn't working, let's check the route definition. Can you show me the current code for your `@rt` decorated `get_workers` function? 

Also, try running this to see what routes are registered:

```python
app.routes
```

This will show us if the route is properly set up.

Still nothing.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let me try a different approach. Can you run this code to test if the route is working:

```python
get_workers('plumber')
```

This calls the function directly. Do you see the worker cards output?

In [ ]:
srv = JupyUvi(app=app, host='0.0.0.0', port=8000)

In [ ]:
srv